In [1]:
%cd .. 

/Data/Projects/Research_ThyroidFNA_ClassAI


/Data/Projects/venv_manager/ai_work#311/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms
from src.models.efficient_net import H0_EfficientNetB0
from src.models.ann import H39_97_ANN


class Heffann3997(nn.Module):
    def __init__(self):
        super(Heffann3997, self).__init__()
        self.efficient_net = H0_EfficientNetB0()
        self.ann = H39_97_ANN()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.transforms = transforms.Compose(
            [transforms.Resize((224, 224)), transforms.ToTensor()]
        )

    def load_weight(self, module_1_path: str, module_2_path: str):
        self.efficient_net.load_state_dict(
            torch.load(module_1_path, map_location=self.device)
        )
        self.ann.load_state_dict(torch.load(module_2_path, map_location=self.device))

    def eval_mode(self):
        self.eval()
        # self.efficient_net.eval()
        # self.ann.eval()

    def forward(self, x):
        """x is tensor of shape (batch_size, 3, 224, 224)"""
        if x.size(0) % 13 != 0:
            raise ValueError("batch_size must be multiple of 13")
        # print(x.shape)
        x = self.efficient_net(x)  # shape (batch_size, 3)
        # print(x.shape)
        x = x.view(-1, 39)  # shape (batch_size // 13, 13 * 3)
        # print(x.shape)
        x = self.ann(x)  # shape (batch_size // 13, 3)
        # print(x.shape)
        return x

    def predict_from_path(self, imgs_path):
        inputs = []
        for img_path in imgs_path:
            img = Image.open(img_path).convert("RGB").resize((1024, 768))
            crops = self._crop_image(img)
            crops = [transforms.ToTensor()(crop) for crop in crops]
            img = transforms.Resize((224, 224))(img)
            img = transforms.ToTensor()(img)
            x = torch.stack([img] + crops)
            inputs.append(x)
        inputs = torch.stack(inputs)
        inputs = inputs.to(self.device)
        inputs = inputs.view(-1, 3, 224, 224)
        outputs = self.forward(inputs)
        return outputs

    def predict_from_PIL(self, imgs_PIL):
        """Make sure that imgs is a list of PIL images and img is RGB"""
        inputs = []
        for img in imgs_PIL:
            crops = self._crop_image(img.resize((1024, 768)))
            crops = [transforms.ToTensor()(crop) for crop in crops]
            img = transforms.Resize((224, 224))(img)
            img = transforms.ToTensor()(img)
            x = torch.stack([img] + crops)
            inputs.append(x)
        inputs = torch.stack(inputs)
        inputs = inputs.to(self.device)
        outputs = self.forward(inputs)
        return outputs

    def _crop_image(self, img):
        img_crops = []
        width, height = img.size
        crop_width = 256
        crop_height = 256
        for i in range(3):  # 3 rows
            for j in range(4):  # 4 columns
                left = j * crop_width
                top = i * crop_height
                right = min(left + crop_width, width)
                bottom = min(top + crop_height, height)
                crop = img.crop((left, top, right, bottom)).resize((224, 224))
                img_crops.append(crop)
        return img_crops

In [9]:
model = Heffann3997()
model.eval_mode()

In [12]:
imgs_path = [
    "/Data/Projects/Research_ThyroidFNA_ClassAI/notebooks/Validation Set (SigReLU).png",
    "/Data/Projects/Research_ThyroidFNA_ClassAI/notebooks/Training Set.png",
]

In [13]:
output = model.predict_from_path(imgs_path)

torch.Size([26, 3, 224, 224])
torch.Size([26, 3])
torch.Size([2, 39])
torch.Size([2, 3])


In [14]:
print(output)

tensor([[ 0.0482,  0.0653, -0.0512],
        [ 0.0482,  0.0653, -0.0512]], grad_fn=<AddmmBackward0>)
